In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ast
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [21]:
# ----------------------------
# LOAD DATA
# ----------------------------
df = pd.read_csv('/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv')
# fix wrong paths in CSV
df['filepath'] = df['filepath'].str.replace(
    '../input/ocular-disease-recognition-odir5k/ODIR-5K/Training Images',
    '/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images',
    regex=False
)

df = df[df['filepath'].apply(os.path.exists)].reset_index(drop=True)


# =========================================================
# 3. CLEAN LABELS COLUMN (['C'] -> 'C')
# =========================================================
df['labels'] = df['labels'].apply(lambda x: ast.literal_eval(x)[0])


# =========================================================
# 4. LABEL ENCODING
# =========================================================
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['labels'])

# number of classes
NUM_CLASSES = len(le.classes_)

# One-hot encoding for softmax
df['target_onehot'] = df['label_encoded'].apply(
    lambda x: tf.keras.utils.to_categorical(x, NUM_CLASSES)
)



In [22]:

# =========================================================
# 5. TRAIN-TEST SPLIT
# =========================================================
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label_encoded']
)


In [23]:
# =========================================================
# 6. TF DATA PIPELINE
# =========================================================
IMG_SIZE = 224
BATCH_SIZE = 16

def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img, label

def make_dataset(df, shuffle=True):
    paths = df['filepath'].values
    labels = np.stack(df['target_onehot'].values)
    
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_dataset(train_df)
test_ds  = make_dataset(test_df, shuffle=False)


In [26]:
# =========================================================
# 7. BUILD EFFICIENTNETB0 MODEL
# =========================================================
base = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base.trainable = True   # Freeze feature extractor

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
out = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(base.input, out)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_6         │ (None, 224, 224,  │          0 │ input_layer_3[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_3     │ (None, 224, 224,  │          7 │ rescaling_6[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_7         │ (None, 224, 224,  │          0 │ normalization_3[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_7[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,059,819 (15.49 MB)

 Trainable params: 4,017,796 (15.33 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [ ]:
# =========================================================
# 8. TRAIN MODEL
# =========================================================
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=50
)


Epoch 1/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 42s 112ms/step - accuracy: 0.7184 - loss: 0.7409 - val_accuracy: 0.4496 - val_loss: 192.7751
Epoch 2/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 45s 121ms/step - accuracy: 0.7673 - loss: 0.6187 - val_accuracy: 0.2518 - val_loss: 136.4775
Epoch 3/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 41s 108ms/step - accuracy: 0.8049 - loss: 0.5273 - val_accuracy: 0.4496 - val_loss: 13.8739
Epoch 4/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 45s 123ms/step - accuracy: 0.8414 - loss: 0.4262 - val_accuracy: 0.4480 - val_loss: 143.0138
Epoch 5/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 46s 124ms/step - accuracy: 0.8899 - loss: 0.3024 - val_accuracy: 0.0461 - val_loss: 5.1397
Epoch 6/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 41s 110ms/step - accuracy: 0.8933 - loss: 0.2921 - val_accuracy: 0.0774 - val_loss: 4.0725
Epoch 7/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 45s 123ms/step - accuracy: 0.9202 - loss: 0.2208 - val_accuracy: 0.0461 - val_loss: 6.6284
Epoch 8/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 41s 109ms/step - accuracy: 0.9451 - 

In [17]:
import os

missing = []

for p in df['filepath']:
    if not os.path.exists(p):
        missing.append(p)

len(missing), missing[:10]

(0, [])

In [18]:
/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images/

SyntaxError: invalid decimal literal (468583224.py, line 1)